In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b34vxnnb
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-b34vxnnb
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4307 sha256=a0586dab27f71ea7979f377e1fae89461d4a4ae5facb33b9daa337373fee27d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-yp6hfdy8/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include<bits/stdc++.h>
using namespace std;

int main(){
  string fname="data.csv";
  string line,word;
  fstream file(fname,ios::in);
  cout << "Hello World\n";
  return 0;
}

Hello World



In [ ]:
%%cu
#include<bits/stdc++.h>
using namespace std;

#define N 2000*2000    
#define M 2000   

double x0[M][M];
double x1[M][M];
double x3[M][M];
double x4[M][M];
double x6[M][M];

double u0[M];
double u1[M];
double u3[M];
double u4[M];
double u6[M];

double vec[M];

__global__ void mat_red(double *a, double *b, double *c){
    int row = blockIdx.y*blockDim.y + threadIdx.y;      
    int column = blockIdx.x*blockDim.x + threadIdx.x;   
    double sum = 0.00;
    if(row < M && column < M){
        for(int i=0; i<M; i++){
            sum += a[row*M + i] * b[i*M + column];    
        }
        c[row*M + column] = sum/M;
    }
}

__global__ void mat_shift(double *b, double *c){
    int row = blockIdx.y*blockDim.y + threadIdx.y;      
    int column = blockIdx.x*blockDim.x + threadIdx.x;   
    if(row < M && column < M){
        b[row*M + column]-=c[column];    
    }
}

__global__ void mat_cov(double *a, double *b, double *c){
    int row = blockIdx.y*blockDim.y + threadIdx.y;      
    int column = blockIdx.x*blockDim.x + threadIdx.x;   
    double sum = 0.00;
    if(row < M && column < M){
        for(int i=0; i<M; i++){
            sum += a[row*M + i] * b[i*M + column];    
        }
        c[row*M + column] = sum/(M-1);
    }
}

__global__ void mat_T(double *a, double *b){ 
    int row = blockIdx.y*blockDim.y + threadIdx.y;     
    int column = blockIdx.x*blockDim.x + threadIdx.x;  
    if(row < M && column < M){
        a[row*M + column]=b[column*M + row];
    }
}

__global__ void mat_dot(double *a, double *b, double *c){  
    int row = blockIdx.y*blockDim.y + threadIdx.y;      
    int column = blockIdx.x*blockDim.x + threadIdx.x;   
    double sum = 0.00;
    if(row < M && column < 1){
        for(int i=0; i<M; i++){
            sum += a[row*M + i] * b[i*M + column];
        }
        c[row*M + column] = sum;
    }
}

__global__ void vec_dot(double *a, double *b, double *c){  
    int row = blockIdx.y*blockDim.y + threadIdx.y;      
    int column = blockIdx.x*blockDim.x + threadIdx.x;   
    double sum = 0.00;
    if(row < 1 && column < 1){
        for(int i=0; i<M; i++){
            sum += a[row*M + i] * b[i*M + column];
        }
        c[row*M + column] = sum;
    }
}

int main(){
    for(int i=0;i<M;i++){
        for(int j=0;j<M;j++){
            x0[i][j]=rand()%40;
            x1[i][j]=rand()%40;
            x3[i][j]=rand()%40;
            x4[i][j]=rand()%40;
            x6[i][j]=rand()%40;
        }
    }
    double Max=DBL_MIN;
    int label=-1;
    for(int i=0;i<M;i++){
        vec[i]=x6[0][i];
    }

    double *a, *b, *c;             
    double *d_a, *d_b, *d_c;      
    int size = N * sizeof(double);

    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    a = (double *)malloc(size);
    b = (double *)malloc(size);
    c = (double *)malloc(size);

    int thread_count[] = {2,4,6,8,10,12,16,24,32,64,128,256,512};

    for(int k=0; k<13; k++){
        int threads=thread_count[k];                
        int blocks = (M+threads-1)/threads;        
        dim3 THREADS(threads, threads);
        dim3 BLOCKS(blocks, blocks);

        float time_elapsed_milliseconds;
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
        cudaEventRecord(start,0);

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=x0[i][j];
                a[i * M + j]=1;
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_red<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);
        
        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        
        for(int i=0;i<M;i++){
            u0[i]=c[i];
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_shift<<<BLOCKS, THREADS>>>(d_b, d_c);

        cudaMemcpy(b, d_b, size, cudaMemcpyDeviceToHost);
        // x0 in b

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_b);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // x0t in a

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_cov<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        // cov0 in c

        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_c);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // icov0 in a

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[i]-u0[i];
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_dot<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[j];
            }
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        vec_dot<<<BLOCKS, THREADS>>>(d_b, d_c, d_a);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // distance in a[0]
        // cout<<a[0]<<endl;
        if(a[0]>Max){
            Max=a[0];
            label=0;
        }

        
        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=x1[i][j];
                a[i * M + j]=1;
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_red<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);
        
        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        
        for(int i=0;i<M;i++){
            u1[i]=c[i];
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_shift<<<BLOCKS, THREADS>>>(d_b, d_c);

        cudaMemcpy(b, d_b, size, cudaMemcpyDeviceToHost);
        // x1 in b

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_b);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // x1t in a

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_cov<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        // cov1 in c

        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_c);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // icov1 in a

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[i]-u0[i];
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_dot<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[j];
            }
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        vec_dot<<<BLOCKS, THREADS>>>(d_b, d_c, d_a);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // distance in a[0]
        // cout<<a[0]<<endl;
        if(a[0]>Max){
            Max=a[0];
            label=1;
        }
        

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=x3[i][j];
                a[i * M + j]=1;
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_red<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);
        
        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        
        for(int i=0;i<M;i++){
            u3[i]=c[i];
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_shift<<<BLOCKS, THREADS>>>(d_b, d_c);

        cudaMemcpy(b, d_b, size, cudaMemcpyDeviceToHost);
        // x3 in b

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_b);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // x3t in a

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_cov<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        // cov3 in c

        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_c);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // icov3 in a

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[i]-u6[i];
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_dot<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[j];
            }
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        vec_dot<<<BLOCKS, THREADS>>>(d_b, d_c, d_a);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // distance in a[0]
        // cout<<a[0]<<endl;
        if(a[0]>Max){
            Max=a[0];
            label=3;
        }


        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=x4[i][j];
                a[i * M + j]=1;
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_red<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);
        
        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        
        for(int i=0;i<M;i++){
            u4[i]=c[i];
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_shift<<<BLOCKS, THREADS>>>(d_b, d_c);

        cudaMemcpy(b, d_b, size, cudaMemcpyDeviceToHost);
        // x4 in b

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_b);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // x4t in a

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_cov<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        // cov4 in c

        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_c);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // icov4 in a

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[i]-u0[i];
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_dot<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[j];
            }
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        vec_dot<<<BLOCKS, THREADS>>>(d_b, d_c, d_a);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // distance in a[0]
        // cout<<a[0]<<endl;
        if(a[0]>Max){
            Max=a[0];
            label=4;
        }


        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=x6[i][j];
                a[i * M + j]=1;
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_red<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);
        
        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        
        for(int i=0;i<M;i++){
            u6[i]=c[i];
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_shift<<<BLOCKS, THREADS>>>(d_b, d_c);

        cudaMemcpy(b, d_b, size, cudaMemcpyDeviceToHost);
        // x6 in b

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_b);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // x6t in a

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_cov<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
        // cov6 in c

        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        mat_T<<<BLOCKS, THREADS>>>(d_a, d_c);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // icov6 in a

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[i]-u0[i];
            }
        }

        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

        mat_dot<<<BLOCKS, THREADS>>>(d_a, d_b, d_c);

        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

        for(int i=0;i<M;i++){
            for(int j=0;j<M;j++){
                b[i * M + j]=vec[j];
            }
        }

        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
        cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);

        vec_dot<<<BLOCKS, THREADS>>>(d_b, d_c, d_a);

        cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);
        // distance in a[0]
        // cout<<a[0]<<endl;
        if(a[0]>Max){
            Max=a[0];
            label=6;
        }

        cudaEventRecord(stop,0);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&time_elapsed_milliseconds, start,stop);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
        time_elapsed_milliseconds = time_elapsed_milliseconds/1000;
        cout <<"time: "<< time_elapsed_milliseconds<<endl;
    }

    // cout<<"Class label of the test sequence is: "<<label<<endl;

    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

time: 6.86
time: 2.45065
time: 2.4258
time: 1.74086
time: 1.89988
time: 1.75416
time: 1.5273
time: 1.55547
time: 1.52225
time: 0.798186
time: 0.804836
time: 0.79674
time: 0.804982

